In [ ]:
!pip install transformers

# ====================================================
# Library
# ====================================================
import os
import sys

import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings
import random
import time

from collections import defaultdict

warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
print(torch.__version__)

2.6.0+cu124


In [ ]:
# ====================================================
# CFG
# ====================================================
# CFGの設定を行う
class CFG:
  model = "bert-base-cased"
  epochs = 5
  batch_size = 16
  seed = 42
  num_labels = 2
  max_len = 160
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set seed
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(CFG.seed)


In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import csv

def merge_txt_and_labels(mail_path, label_csv_path, output_csv_path):
    # メール本文を格納する辞書
    email_contents = {}

    # フォルダ内のすべての .txt ファイルを取得し、内容を辞書に格納
    txt_files = sorted([f for f in os.listdir(mail_path) if f.endswith('.txt')])
    for txt_file in txt_files:
        file_path = os.path.join(mail_path, txt_file)
        with open(file_path, 'r', encoding='utf-8') as f:
            email_contents[txt_file] = f.read()

    # ラベルCSVを読み込み、本文と統合
    merged_data = []
    with open(label_csv_path, 'r', encoding='utf-8') as label_csv:
        reader = csv.DictReader(label_csv, delimiter='\t')
        for row in reader:
            filename = row['file_name']
            target = row['label']
            if filename in email_contents:
                # filenameを含まず、targetとcontentのみを格納
                merged_data.append({'target': target, 'content': email_contents[filename]})

    # 統合データをCSVに書き込み
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as output_csv:
        fieldnames = ['target', 'content']  # filenameを除外
        writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(merged_data)

    print(f"統合データが '{output_csv_path}' に保存されました。")

# 入力フォルダとCSVパスの指定
mail_folder_path = "/content/drive/MyDrive/train_2000-2585"
label_csv_path = "/content/drive/MyDrive/train_master.tsv"
output_csv_path = "/content/emails.csv"

# 統合処理を実行
merge_txt_and_labels(mail_folder_path, label_csv_path, output_csv_path)


Mounted at /content/drive
統合データが '/content/emails.csv' に保存されました。


In [ ]:
# ====================================================
# Read Data, Split Data
# ====================================================
INPUT = "/content/"
train = pd.read_csv(os.path.join(INPUT, "emails.csv"))

df_train, df_val = train_test_split(
  train,
  test_size=0.2,
  random_state=CFG.seed
)


In [ ]:
# ====================================================
# Dataset
# ====================================================
class MailDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):

        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
                      review,
                      add_special_tokens=True,
                      max_length=self.max_len,
                      truncation=True,  # 必須
                      padding='max_length',  # 推奨
                      return_attention_mask=True,
                      return_tensors='pt',
    )
        return {
              'review_text': review,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten(),
              'targets': torch.tensor(target, dtype=torch.long)
    }


def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = MailDataset(
    reviews=df.content.to_numpy(),
    targets=df.target.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
    return DataLoader(
            ds,
            batch_size=batch_size,
            num_workers=4
          )

tokenizer = BertTokenizer.from_pretrained(CFG.model)

train_data_loader = create_data_loader(df_train, tokenizer, CFG.max_len, CFG.batch_size)
val_data_loader = create_data_loader(df_val, tokenizer, CFG.max_len, CFG.batch_size)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# ====================================================
# Model
# ====================================================
class DisasterClassifier(nn.Module):

    def __init__(self,n_classes):
        super(DisasterClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(CFG.model)
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):

        # Get the pooled output directly, no need for _
        # This assumes BertModel returns a tuple (sequence_output, pooled_output)
        outputs = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask
        )
        pooled_output = outputs[1]  # Access the pooled output tensor

        # Convert pooled_output to a PyTorch Tensor
        # pooled_output = pooled_output.to(device) # Assuming device is your CUDA device
        # This line is not needed, as pooled_output should already be on the correct device

        output = self.drop(pooled_output)
        return  self.out(output)

model = DisasterClassifier(2)
model = model.to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# ====================================================
# Optimizer
# ====================================================
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_data_loader)* CFG.epochs

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)



In [ ]:
# ====================================================
# Function
# ====================================================
def train_epoch( model, dataloader, loss_fn, optimizer, device, scheduler,n_examples):

    model = model.train()

    losses =[]
    correct_pred = 0

    for d in dataloader:
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        targets = d['targets'].to(device)


        outputs = model(
            input_ids =input_ids,
            attention_mask = attention_mask
        )

        _,pred = torch.max(outputs , dim=1)

        loss = loss_fn(outputs, targets)

        correct_pred += torch.sum(pred == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_pred.double() / n_examples, np.mean(losses)


def train_eval( model, dataloader, loss_fn, device,n_examples):

    model = model.eval()

    losses =[]
    correct_pred = 0

    with torch.no_grad():

        for d in dataloader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            targets = d['targets'].to(device)


            outputs = model(
                input_ids =input_ids,
                attention_mask = attention_mask
                )

            _,pred = torch.max(outputs , dim=1)

            loss = loss_fn(outputs, targets)

            correct_pred += torch.sum(pred == targets)

            losses.append(loss.item())

    return correct_pred.double()/ n_examples, np.mean(losses)


In [ ]:
def train_eval( model, dataloader, loss_fn, device,n_examples):

    model = model.eval()

    losses =[]
    correct_pred = 0

    with torch.no_grad():

        for d in dataloader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            targets = d['targets'].to(device)


            outputs = model(
                input_ids =input_ids,
                attention_mask = attention_mask
                )

            _,pred = torch.max(outputs , dim=1)

            loss = loss_fn(outputs, targets)

            correct_pred += torch.sum(pred == targets)

            losses.append(loss.item())

    return correct_pred.double()/ n_examples, np.mean(losses)


In [ ]:
%%time

history = defaultdict(list)
best_acc =0

for epoch in range(CFG.epochs):

    print(f'epochs {epoch+1}/{CFG.epochs}')
    train_acc, train_loss = train_epoch(model,train_data_loader,loss_fn, optimizer, device,scheduler, len(df_train))

    print(f'Train loss {train_loss} accuracy {train_acc}')
    val_acc, val_loss = train_eval(model,val_data_loader,loss_fn,device,len(df_val))

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc.to('cpu').detach().numpy().copy())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.to('cpu').detach().numpy().copy())
    history['val_loss'].append(val_loss)

    if val_acc > best_acc:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_acc = val_acc


epochs 1/5
Train loss 0.5374753028154373 accuracy 0.7051282051282052
Val   loss 0.33154964633286 accuracy 0.8983050847457628

epochs 2/5
Train loss 0.2041708734507362 accuracy 0.9444444444444445
Val   loss 0.1799671738408506 accuracy 0.9322033898305084

epochs 3/5
Train loss 0.059072279231622817 accuracy 0.982905982905983
Val   loss 0.12389584293123335 accuracy 0.9491525423728814

epochs 4/5
Train loss 0.02414779094979167 accuracy 0.9935897435897437
Val   loss 0.1303005606168881 accuracy 0.9576271186440678

epochs 5/5
Train loss 0.015445962133041273 accuracy 0.9957264957264959
Val   loss 0.12416386127006263 accuracy 0.9661016949152542

CPU times: user 52.7 s, sys: 3.77 s, total: 56.5 s
Wall time: 1min 16s


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import os
import pandas as pd

def load_text_files_from_folder(folder_path):
    texts = []
    file_names = [] # List to store filenames
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as f:
                texts.append(f.read())
                file_names.append(file_name) # Append filename to list
    return texts, file_names  # Return both texts and filenames

# Google Driveのフォルダパス
folder_path = '/content/drive/MyDrive/train_0000-1999'
texts, file_names = load_text_files_from_folder(folder_path) # Get texts and filenames

# CFGクラスから設定を取得
tokenizer = BertTokenizer.from_pretrained(CFG.model)  # Use CFG.model

# モデルのロード（CFGクラスの設定を利用） - Use BertModel instead of BertForSequenceClassification
model = DisasterClassifier(CFG.num_labels)  # Use your original model class

# 学習済みモデルのロード
model.load_state_dict(torch.load('best_model_state.bin', map_location=torch.device(CFG.device))) # Use CFG.device

# 評価モードに切り替え
model.to(CFG.device) # Use CFG.device
model.eval()

def predict_spam(texts, model, tokenizer):
    model.eval()

    # Create an instance of CFG to access max_length
    cfg_instance = CFG()

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=cfg_instance.max_len,  # Access max_len from the instance
        return_tensors="pt"
    ).to(cfg_instance.device)  # Also use cfg_instance.device

    with torch.no_grad():
        # Only pass input_ids and attention_mask to the model
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

    predictions = torch.argmax(outputs, dim=1) # Remove .logits
    return predictions

# 迷惑メールの分類を実行
results = predict_spam(texts, model, tokenizer)

# ファイル名と予測結果をソート
sorted_results = sorted(zip(file_names, results.tolist()), key=lambda x: x[0])

# TSVファイルに出力
df = pd.DataFrame(sorted_results, columns=['file_name', 'label'])
df.to_csv('spam_classification_results.tsv', sep='\t', index=False)

# 結果の表示
for file_name, label in sorted_results:
    print(f"{file_name}: {label}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.68 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.67 GiB is free. Process 11427 has 12.07 GiB memory in use. Of the allocated memory 8.28 GiB is allocated by PyTorch, and 3.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)